In [2]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install captum
!pip install transformers==3

1.13.0+cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 49.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 31.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 754.6/754.6 KB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 62.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 104.6 MB/s

In [3]:
from google.colab import files
uploaded = files.upload()

Saving gcn_2l_model.py to gcn_2l_model.py


In [35]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool
import matplotlib.pyplot as plt
import torch
from torch_geometric.datasets import TUDataset
import numpy as np
from termcolor import colored
from torchsummary import summary
from torch.autograd import Variable
from keras import backend as K
from statistics import mean
from sklearn import metrics
from copy import deepcopy
from captum.attr import Saliency
from scipy.spatial.distance import hamming
from itertools import zip_longest
from time import perf_counter
import csv
import scipy
import torch.nn as nn
import torch_geometric.nn as gnn
from torch import Tensor
from torch_geometric.typing import OptPairTensor, Adj, OptTensor, Size
from typing import Callable, Union, Tuple
from torch_sparse import SparseTensor
from time import perf_counter
import random
import pandas
from torch_geometric.loader import DataLoader
import sklearn
import gcn_2l_model
#import model_train_test_gc
#from model_train_test_gc import Model_Train_Test

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **SA GC Object Oriented Programming**

In [49]:
class SA_GC(object):
  def __init__(self, task, method, graph, importance_range, start_epoch, input_dim, hid_dim, output_dim):

    self.GCN_model = self.load_model(task, method, load_index=start_epoch, input_dim=input_dim, hid_dim=hid_dim, output_dim=output_dim)
    self.criterion = torch.nn.CrossEntropyLoss()
    self.importance_dict = {}
    self.new_graph, self.saliency_maps = self.drop_important_nodes(self.GCN_model, graph, importance_range)
    


  def load_model(self, task, method, load_index, input_dim, hid_dim, output_dim):

    if load_index != 0:
      GCN_model, optimizer, load_index = self.loading_config(task, method, load_index, input_dim, hid_dim, output_dim)
      return GCN_model
    else:
      GCN_model = gcn_2l_model.GCN_2Layer_Model(model_level='graph', dim_node=input_dim, dim_hidden=hid_dim, dim_output=output_dim)
      return GCN_model


  def loading_config(self, task, method, load_index, input_dim, hid_dim, output_dim):
    GCN_model = gcn_2l_model.GCN_2Layer_Model(model_level='graph', dim_node=input_dim, dim_hidden=hid_dim, dim_output=output_dim)
    optimizer = torch.optim.Adam(params = GCN_model.parameters(), lr=0.001)
    checkpoint = torch.load("/content/drive/My Drive/Explainability Methods/"+str(method)+" on " + str(task) + "/Model/" + str(method) + " on " + str(task) + " classifier model_" + str(load_index)+".pt")
    GCN_model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    
    return GCN_model, optimizer, epoch
  

  def loss_calculations(self, preds, gtruth):
    loss_per_epoch = self.criterion(preds, gtruth)
    return loss_per_epoch

  def remove_nones(self, sample_grads):
    sample_grads2 = []
    for item in sample_grads:
      Each_Graph = []
      for item2 in item:
        if item2 != None:
          Each_Graph.append(torch.tensor(item2.clone().detach().requires_grad_(True),requires_grad=True))
        else:
          Each_Graph.append(torch.tensor(0))
      sample_grads2.append(Each_Graph)

    return sample_grads2

  def compute_grad(self, model, graph, with_respect):
    post_conv1, post_conv2, out_readout, prediction = model(graph.x, graph.edge_index, graph.batch)
    if with_respect == 1 :
      loss = self.loss_calculations(prediction, graph.y)
      #print(loss)
    elif with_respect == 2:
      loss = self.loss_calculations(prediction, torch.tensor([0]))
      #print(loss)  
    elif with_respect == 3:
      loss = self.loss_calculations(prediction, torch.tensor([1]))
      #print(loss)
    return torch.autograd.grad(loss, list(self.GCN_model.parameters()), allow_unused=True)

  def compute_sample_grads(self, model, test_dataset, with_respect):

    sample_grads = [self.compute_grad(model, graph, with_respect) for graph in test_dataset]
    sample_grads = self.remove_nones(sample_grads)
    sample_grads = zip(*sample_grads)
    sample_grads = [torch.stack(shards) for shards in sample_grads]
    return sample_grads

  

  def compute_square_gradients(self, model, dataset):
    per_sample_grads_wrt_graph_label = self.compute_sample_grads(model, dataset, 1)
    per_sample_grads_wrt_class_zero = self.compute_sample_grads(model, dataset, 2)
    per_sample_grads_wrt_class_one = self.compute_sample_grads(model, dataset, 3)

    grads_wrt_graph_label = torch.square(per_sample_grads_wrt_graph_label[0])
    #square_grads_wrt_graph_label = torch.square(per_sample_grads_wrt_graph_label[1])
    square_grads_wrt_graph_label = grads_wrt_graph_label.detach().tolist()

    grads_wrt_class_zero = torch.square(per_sample_grads_wrt_class_zero[0])
    #square_grads_wrt_class_zero = torch.square(per_sample_grads_wrt_class_zero[1])
    square_grads_wrt_class_zero = grads_wrt_class_zero.detach().tolist()

    grads_wrt_class_one = torch.square(per_sample_grads_wrt_class_one[0])
    #square_grads_wrt_class_one = torch.square(per_sample_grads_wrt_class_one[1])
    square_grads_wrt_class_one = grads_wrt_class_one.detach().tolist()

    return square_grads_wrt_graph_label, square_grads_wrt_class_zero, square_grads_wrt_class_one
  

  def saliency(self, dataset, gradients):
    Final= []
    for i in range(len(dataset)):
      Mid = []
      for node in dataset[i].x.detach().numpy():
        First = []
        for grad_list in gradients[i]:
          First.append(np.multiply(node, grad_list))
        Mid.append(First)
      Final.append(Mid)
    
    Saliency_Nodes = []
    for graph in Final:
      Node = []
      for node in graph:
        Grad = []
        for grad in node:
          Grad.append(sum(grad))
        Node.append(sum(Grad))
      #norm = [(float(i)-min(Node))/(max(Node)-min(Node)) for i in Node]
      norm = [(float(i))/(max(Node) + 1e-16) for i in Node]
      Saliency_Nodes.append(norm)
    return Saliency_Nodes
  


  def is_salient(self, index, score, importance_range):
    start, end = importance_range
    if start <= score <= end:
      return True
    else:

      return False
  

  def drop_important_nodes(self, model, graph, importance_range):
    square_grads_wrt_graph_label, square_grads_wrt_class_zero, square_grads_wrt_class_one = self.compute_square_gradients(model, graph)
    print(len(square_grads_wrt_graph_label))
    SA_attribution_scores = self.saliency(graph, square_grads_wrt_graph_label)
    occluded_GNNgraph_list = []
    
    for i in range(len(SA_attribution_scores)):
      sample_graph = deepcopy(graph[i])
      graph_dict ={}
      for j in range(len(sample_graph.x)):
        
        if self.is_salient(j, (SA_attribution_scores[i][j]), importance_range):
          #print("before: ", sample_graph.x[j])
          sample_graph.x[j][:] = 0
          #print(torch.zeros_like(sample_graph.x[j]))
          #print("manipulated: ",sample_graph.x[j])
          graph_dict[j] = True
        else:
          graph_dict[j] = False
      self.importance_dict[i] = graph_dict
      occluded_GNNgraph_list.append(sample_graph)


    return occluded_GNNgraph_list, SA_attribution_scores



dataset = TUDataset(root='data/TUDataset', name='MUTAG')



#new_output = SA_GC(graph=[graph], importance_range=(0.5, 1), start_epoch=200, input_dim = len(graph.x[0]), hid_dim = 7, output_dim = 2)
new_output = SA_GC(task="Graph Classification", method="SA", graph=[dataset[0]], importance_range=(0.5, 1), start_epoch=200, input_dim = len(dataset[0].x[0]), hid_dim = 7, output_dim = 2)
print(new_output.new_graph[-1].x, dataset[-1].x)
print(new_output.saliency_maps)
print(new_output.importance_dict)


1
tensor([[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.]]) tensor([[1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [

<ipython-input-49-decf5eda2cf6>:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Each_Graph.append(torch.tensor(item2.clone().detach().requires_grad_(True),requires_grad=True))


In [ ]:
dataset = TUDataset(root='data/TUDataset', name='MUTAG')



#new_output = SA_GC(graph=[graph], importance_range=(0.5, 1), start_epoch=200, input_dim = len(graph.x[0]), hid_dim = 7, output_dim = 2)
new_output = SA_GC(task="Graph Classification", method="SA", graph=dataset, importance_range=(0.5, 1), start_epoch=200, input_dim = len(dataset[0].x[0]), hid_dim = 7, output_dim = 2)
print(new_output.new_graph[-1].x, dataset[-1].x)

print(new_output.importance_dict)

## **GuidedBP GC Object Oriented Programming**

In [47]:
class GuidedBP_GC(object):
  def __init__(self, task, method, graph, importance_range, load_index, input_dim, hid_dim, output_dim):

    self.GCN_model = self.load_model(task, method, load_index=load_index, input_dim=input_dim, hid_dim=hid_dim, output_dim=output_dim)
    self.criterion = torch.nn.CrossEntropyLoss()
    self.importance_dict = {}
    self.new_graph, self.saliency_maps = self.drop_important_nodes(self.GCN_model, graph, importance_range)
    


  def load_model(self, task, method, load_index, input_dim, hid_dim, output_dim):

    if load_index != 0:
      GCN_model, optimizer, load_index = self.loading_config(task, method, load_index, input_dim, hid_dim, output_dim)
      return GCN_model
    else:
      GCN_model = gcn_2l_model.GCN_2Layer_Model(model_level='graph', dim_node=input_dim, dim_hidden=hid_dim, dim_output=output_dim)
      return GCN_model


  def loading_config(self, task, method, load_index, input_dim, hid_dim, output_dim):
    GCN_model = gcn_2l_model.GCN_2Layer_Model(model_level='graph', dim_node=input_dim, dim_hidden=hid_dim, dim_output=output_dim)
    optimizer = torch.optim.Adam(params = GCN_model.parameters(), lr=1e-4)
    checkpoint = torch.load("/content/drive/My Drive/Explainability Methods/"+str(method)+" on " + str(task) + "/Model/" + str(method) + " on " + str(task) + " classifier model_" + str(load_index)+".pt")
    GCN_model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    
    return GCN_model, optimizer, epoch
  

  def loss_calculations(self, preds, gtruth):
    loss_per_epoch = self.criterion(preds, gtruth)
    return loss_per_epoch

  def remove_nones(self, sample_grads):
    sample_grads2 = []
    for item in sample_grads:
      Each_Graph = []
      for item2 in item:
        if item2 != None:
          Each_Graph.append(torch.tensor(item2.clone().detach().requires_grad_(True),requires_grad=True))
        else:
          Each_Graph.append(torch.tensor(0))
      sample_grads2.append(Each_Graph)

    return sample_grads2

  def compute_grad(self, model, graph, with_respect):
    post_conv1, post_conv2, out_readout, prediction = model(graph.x, graph.edge_index, graph.batch)
    if with_respect == 1 :
      loss = self.loss_calculations(prediction, graph.y)
      #print(loss)
    elif with_respect == 2:
      loss = self.loss_calculations(prediction, torch.tensor([0]))
      #print(loss)  
    elif with_respect == 3:
      loss = self.loss_calculations(prediction, torch.tensor([1]))
      #print(loss)
    return torch.autograd.grad(loss, list(self.GCN_model.parameters()), allow_unused=True)

  def compute_sample_grads(self, model, test_dataset, with_respect):

    sample_grads = [self.compute_grad(model, graph, with_respect) for graph in test_dataset]
    sample_grads = self.remove_nones(sample_grads)
    sample_grads = zip(*sample_grads)
    sample_grads = [torch.stack(shards) for shards in sample_grads]
    return sample_grads

  

  def compute_guided_gradients(self, your_model, dataset):
    per_sample_grads_wrt_graph_label = self.compute_sample_grads(your_model, dataset, 1)
    per_sample_grads_wrt_class_zero = self.compute_sample_grads(your_model, dataset, 2)
    per_sample_grads_wrt_class_one = self.compute_sample_grads(your_model, dataset, 3)

    grads_wrt_graph_label = per_sample_grads_wrt_graph_label[0]
    guided_grads_wrt_graph_label = torch.maximum(torch.zeros_like(grads_wrt_graph_label), grads_wrt_graph_label)
    guided_grads_wrt_graph_label = guided_grads_wrt_graph_label.detach().tolist()

    grads_wrt_class_zero = per_sample_grads_wrt_class_zero[0]
    guided_grads_wrt_class_zero = torch.maximum(torch.zeros_like(grads_wrt_class_zero), grads_wrt_class_zero)
    guided_grads_wrt_class_zero = guided_grads_wrt_class_zero.detach().tolist()

    grads_wrt_class_one = per_sample_grads_wrt_class_one[0]
    guided_grads_wrt_class_one = torch.maximum(torch.zeros_like(grads_wrt_class_one), grads_wrt_class_one)
    guided_grads_wrt_class_one = guided_grads_wrt_class_one.detach().tolist()

    return guided_grads_wrt_graph_label, guided_grads_wrt_class_zero, guided_grads_wrt_class_one
  

  def saliency(self, dataset, gradients):
    Final= []
    for i in range(len(dataset)):
      Mid = []
      for node in dataset[i].x.detach().numpy():
        First = []
        for grad_list in gradients[i]:
          First.append(np.multiply(node, grad_list))
        Mid.append(First)
      Final.append(Mid)
    
    Saliency_Nodes = []
    for graph in Final:
      Node = []
      for node in graph:
        Grad = []
        for grad in node:
          Grad.append(sum(grad))
        Node.append(sum(Grad))
      #norm = [(float(i)-min(Node))/(max(Node)-min(Node)) for i in Node]
      norm = [(float(i))/(max(Node) + 1e-16) for i in Node]
      Saliency_Nodes.append(norm)
    return Saliency_Nodes
  


  def is_salient(self, index, score, importance_range):
    start, end = importance_range
    if start <= score <= end:
      return True
    else:

      return False
  

  def drop_important_nodes(self, model, graph, importance_range):
    square_grads_wrt_graph_label, square_grads_wrt_class_zero, square_grads_wrt_class_one = self.compute_guided_gradients(model, graph)
    print(len(square_grads_wrt_graph_label))
    GuidedBP_attribution_scores = self.saliency(graph, square_grads_wrt_graph_label)
    occluded_GNNgraph_list = []
    
    for i in range(len(GuidedBP_attribution_scores)):
      sample_graph = deepcopy(graph[i])
      graph_dict ={}
      for j in range(len(sample_graph.x)):
        
        if self.is_salient(j, (GuidedBP_attribution_scores[i][j]), importance_range):
          #print("before: ", sample_graph.x[j])
          sample_graph.x[j][:] = 0
          #print(torch.zeros_like(sample_graph.x[j]))
          #print("manipulated: ",sample_graph.x[j])
          graph_dict[j] = True
        else:
          graph_dict[j] = False
      self.importance_dict[i] = graph_dict
      occluded_GNNgraph_list.append(sample_graph)


    return occluded_GNNgraph_list, GuidedBP_attribution_scores



dataset = TUDataset(root='data/TUDataset', name='MUTAG')



#new_output = SA_GC(graph=[graph], importance_range=(0.5, 1), start_epoch=200, input_dim = len(graph.x[0]), hid_dim = 7, output_dim = 2)
new_output = GuidedBP_GC(task="Graph Classification", method="GuidedBP", graph=[dataset[0]], importance_range=(0.5, 1), load_index=200, input_dim = len(dataset[0].x[0]), hid_dim = 7, output_dim = 2)
print(new_output.new_graph[-1].x, dataset[-1].x)
print(new_output.saliency_maps)
print(new_output.importance_dict)


1
tensor([[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.]]) tensor([[1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [

<ipython-input-47-da95ae09e176>:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Each_Graph.append(torch.tensor(item2.clone().detach().requires_grad_(True),requires_grad=True))


## **CAM GC Object Oriented Programming**

In [44]:
class CAM_GC(object):
  def __init__(self, task, method, graph, importance_range, load_index, input_dim, hid_dim, output_dim):

    self.GCN_model = self.load_model(task, method, load_index=load_index, input_dim=input_dim, hid_dim=hid_dim, output_dim=output_dim)
    self.criterion = torch.nn.CrossEntropyLoss()
    self.importance_dict = {}
    self.new_graph, self.saliency_maps = self.drop_important_nodes(self.GCN_model, graph, importance_range)
    


  def load_model(self, task, method, load_index, input_dim, hid_dim, output_dim):

    if load_index != 0:
      GCN_model, optimizer, load_index = self.loading_config(task, method, load_index, input_dim, hid_dim, output_dim)
      return GCN_model
    else:
      GCN_model = gcn_2l_model.GCN_2Layer_Model(model_level='graph', dim_node=input_dim, dim_hidden=hid_dim, dim_output=output_dim)
      return GCN_model


  def loading_config(self, task, method, load_index, input_dim, hid_dim, output_dim):
    GCN_model = gcn_2l_model.GCN_2Layer_Model(model_level='graph', dim_node=input_dim, dim_hidden=hid_dim, dim_output=output_dim)
    optimizer = torch.optim.Adam(params=GCN_model.parameters(), lr=0.001)
    checkpoint = torch.load("/content/drive/My Drive/Explainability Methods/"+str(method)+" on " + str(task) + "/Model/" + str(method) + " on " + str(task) + " classifier model_" + str(load_index)+".pt")
    GCN_model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    
    return GCN_model, optimizer, epoch
  

  def loss_calculations(self, preds, gtruth):
    loss_per_epoch = self.criterion(preds, gtruth)
    return loss_per_epoch

  def weights_of_model(self, model):
    Dense_Layer_Weights = model.ffn.weight
    #Dense_Layer_Biases = model.lin.bias.data

    #GConv3_Layer_Weights = model.conv3.lin.weight.data
    GConv2_Layer_Weights = model.gconv2.weight
    GConv1_Layer_Weights = model.gconv1.weight

    Dense_Layer_Weights = Dense_Layer_Weights.detach().tolist()
    #GConv3_Layer_Weights = GConv3_Layer_Weights.detach().tolist()
    GConv2_Layer_Weights = GConv2_Layer_Weights.detach().tolist()
    GConv1_Layer_Weights = GConv1_Layer_Weights.detach().tolist()

    #return GConv1_Layer_Weights, GConv2_Layer_Weights, GConv3_Layer_Weights, Dense_Layer_Weights
    return GConv1_Layer_Weights, GConv2_Layer_Weights, Dense_Layer_Weights
  
  
  def CAM_FeatureMAP_production(self, model, your_dataset):
    index_of_winner_labels = []
    FeatureMaps_of_the_Last_Conv = []
    output_of_the_GAP = []
    Final_predictions = []

    model.eval()
    for batched_data in your_dataset:
      print(batched_data.x)
      CAM_Test_One_Before_Last_Conv, CAM_Test_Last_Conv, CAM_Test_GAP, CAM_output = model(batched_data.x, batched_data.edge_index, batched_data.batch)
      #index_of_winner_label = CAM_output.argmax(dim=1) 
      logits = F.log_softmax(CAM_output, dim=1)
      prob = F.softmax(logits, dim=1)
      index_of_winner_label = prob.argmax(dim=1) 

      index_of_winner_labels.append(index_of_winner_label.detach().tolist()) 
      Final_predictions.append(prob.detach().tolist())
      output_of_the_GAP.append(CAM_Test_GAP.detach().tolist())
      FeatureMaps_of_the_Last_Conv.append(CAM_Test_Last_Conv.detach().tolist())
    return FeatureMaps_of_the_Last_Conv, output_of_the_GAP, Final_predictions, index_of_winner_labels



  def weight_wrt_class_and_performance(self, index_of_winner_labels, Dense_Layer_Weights):
    Weights_of_the_Predicted_Class = []
    for i in range(len(index_of_winner_labels)):
      Weights_of_the_Predicted_Class.append(Dense_Layer_Weights[index_of_winner_labels[i][0]])
    
    Weights_of_the_Class_0 = []
    for i in range(len(index_of_winner_labels)):
      Weights_of_the_Class_0.append(Dense_Layer_Weights[0])

    Weights_of_the_Class_1 = []
    for i in range(len(index_of_winner_labels)):
      Weights_of_the_Class_1.append(Dense_Layer_Weights[1])

    return Weights_of_the_Predicted_Class, Weights_of_the_Class_0, Weights_of_the_Class_1



  def CAM_Attribution_Scores(self, model, your_dataset):
    GConv1_Layer_Weights, GConv2_Layer_Weights, Dense_Layer_Weights = self.weights_of_model(model)
    FeatureMaps_of_the_Last_Conv, output_of_the_GAP, Final_predictions, index_of_winner_labels = self.CAM_FeatureMAP_production(model, your_dataset)
    Weights_of_the_Predicted_Class, Weights_of_the_Class_0, Weights_of_the_Class_1 = self.weight_wrt_class_and_performance(index_of_winner_labels, Dense_Layer_Weights)
    Weights_and_Maps_Multiplication_on_Nodes_of_each_graph = []
    Normalized_Attributions = []

    for i in range(len(FeatureMaps_of_the_Last_Conv)):
      Each_Graph = []
      for j in range(len(FeatureMaps_of_the_Last_Conv[i])):
        Each_Graph.append(np.multiply(Weights_of_the_Predicted_Class[i], FeatureMaps_of_the_Last_Conv[i][j]))
      Weights_and_Maps_Multiplication_on_Nodes_of_each_graph.append(Each_Graph)

    for i in range(len(FeatureMaps_of_the_Last_Conv)):
      Each_Graph = []
      for j in range(len(FeatureMaps_of_the_Last_Conv[i])):
        Each_Graph.append(sum(Weights_and_Maps_Multiplication_on_Nodes_of_each_graph[i][j]))
      norm = [(float(i))/(max(Each_Graph) + 1e-16) for i in Each_Graph] 
      Normalized_Attributions.append(norm)
    return Normalized_Attributions

  def is_salient(self, score, importance_range):
    start, end = importance_range
    if start <= score <= end:
      return True
    else:
      return False


  def drop_important_nodes(self, your_model, your_dataset, importance_range):
    CAM_attribution_scores = self.CAM_Attribution_Scores(your_model, your_dataset)
    occluded_GNNgraph_list = []
    
    for i in range(len(CAM_attribution_scores)):
      sample_graph = deepcopy(your_dataset[i])
      graph_dict ={}
      for j in range(len(sample_graph.x)):
        
        if self.is_salient((CAM_attribution_scores[i][j]), importance_range):
          #print("before: ", sample_graph.x[j])
          sample_graph.x[j][:] = 0
          #print(torch.zeros_like(sample_graph.x[j]))
          #print("manipulated: ",sample_graph.x[j])
          graph_dict[j] = True
        else:
          graph_dict[j] = False
      self.importance_dict[i] = graph_dict
      occluded_GNNgraph_list.append(sample_graph)
    return occluded_GNNgraph_list, CAM_attribution_scores



  


dataset = TUDataset(root='data/TUDataset', name='MUTAG')



#new_output = SA_GC(graph=[graph], importance_range=(0.5, 1), start_epoch=200, input_dim = len(graph.x[0]), hid_dim = 7, output_dim = 2)
new_output = CAM_GC(task="Graph Classification", method="CAM", graph=[dataset[0]], importance_range=(0.5, 1), load_index=200, input_dim = len(dataset[0].x[0]), hid_dim = 7, output_dim = 2)
print(new_output.new_graph[-1].x, dataset[-1].x)
print(new_output.saliency_maps)
print(new_output.importance_dict)


tensor([[1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.]])
tensor([[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0.

## **Grad-CAM GC Object Oriented Programming**

In [41]:
class Grad_CAM_GC(object):
  def __init__(self, task, method, graph, importance_range, load_index, input_dim, hid_dim, output_dim):

    self.GCN_model = self.load_model(task, method, load_index=load_index, input_dim=input_dim, hid_dim=hid_dim, output_dim=output_dim)
    self.criterion = torch.nn.CrossEntropyLoss()
    self.importance_dict = {}
    self.new_graph, self.saliency_maps = self.drop_important_nodes(self.GCN_model, graph, importance_range)
    


  def load_model(self, task, method, load_index, input_dim, hid_dim, output_dim):

    if load_index != 0:
      GCN_model, optimizer, load_index = self.loading_config(task, method, load_index, input_dim, hid_dim, output_dim)
      return GCN_model
    else:
      GCN_model = gcn_2l_model.GCN_2Layer_Model(model_level='graph', dim_node=input_dim, dim_hidden=hid_dim, dim_output=output_dim)
      return GCN_model


  def loading_config(self, task, method, load_index, input_dim, hid_dim, output_dim):
    GCN_model = gcn_2l_model.GCN_2Layer_Model(model_level='graph', dim_node=input_dim, dim_hidden=hid_dim, dim_output=output_dim)
    optimizer = torch.optim.Adam(params=GCN_model.parameters(), lr=0.001)
    checkpoint = torch.load("/content/drive/My Drive/Explainability Methods/"+str(method)+" on " + str(task) + "/Model/" + str(method) + " on " + str(task) + " classifier model_" + str(load_index)+".pt")
    GCN_model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    
    return GCN_model, optimizer, epoch
  

  def loss_calculations(self, preds, gtruth):
    loss_per_epoch = self.criterion(preds, gtruth)
    return loss_per_epoch

  def compute_grad(self, model, graph, with_respect):
    Grad_CAM_Test_One_Before_Last_Conv, Grad_CAM_Test_Last_Conv, Grad_CAM_Test_GAP, Grad_CAM_Test_out = model(graph.x, graph.edge_index, graph.batch)
    #print(prediction)
    if with_respect == 1 :
      loss = self.loss_calculations(Grad_CAM_Test_out, graph.y)
      #print(loss)
      #print("done")
    elif with_respect == 2:
      loss = self.loss_calculations(Grad_CAM_Test_out, torch.tensor([0]))
      #print(loss)  
    elif with_respect == 3:
      loss = self.loss_calculations(Grad_CAM_Test_out, torch.tensor([1]))
      #print(loss)
    return torch.autograd.grad(loss, list(model.parameters()),allow_unused=True)

  
  def remove_nones(self, sample_grads):
    #print(type(sample_grads[0]))
    sample_grads2 = []
    for item in sample_grads:
      Each_Graph = []
      for item2 in item:
        if item2 != None:
          Each_Graph.append(torch.tensor(item2.clone().detach().requires_grad_(True),requires_grad=True))
        else:
          Each_Graph.append(torch.tensor(0))
      sample_grads2.append(Each_Graph)
      #print("separate                         ")
          #item2 = torch.tensor([0])
    #print(np.shape(sample_grads[0]))
    #print(np.shape(sample_grads2[0]))
    return sample_grads2


  def compute_sample_grads(self, model, test_dataset, with_respect):

    sample_grads = [self.compute_grad(model, graph, with_respect) for graph in test_dataset]
    #print(np.shape(sample_grads[0]))
    #print(sample_grads[20])
    sample_grads = self.remove_nones(sample_grads)
    sample_grads = zip(*sample_grads)
    sample_grads = [torch.stack(shards) for shards in sample_grads]
    #sample_grads = [print(shards) for shards in sample_grads]
    return sample_grads


  def compute_grad_cam_gradients(self, your_model, dataset):
    per_sample_grads_wrt_graph_label = self.compute_sample_grads(your_model, dataset, 1)
    per_sample_grads_wrt_class_zero = self.compute_sample_grads(your_model, dataset, 2)
    per_sample_grads_wrt_class_one = self.compute_sample_grads(your_model, dataset, 3)

    grads_wrt_graph_label = per_sample_grads_wrt_graph_label[2].detach().tolist()
    grads_wrt_class_zero = per_sample_grads_wrt_class_zero[2].detach().tolist()
    grads_wrt_class_one = per_sample_grads_wrt_class_one[2].detach().tolist()


    return grads_wrt_graph_label, grads_wrt_class_zero, grads_wrt_class_one

  def is_salient(self, score, importance_range):
    start, end = importance_range
    if start <= score <= end:
      return True
    else:
      return False


  def Grad_CAM_FeatureMAP_production(self, your_model, test_loader):
    index_of_winner_labels = []
    FeatureMaps_of_the_Last_Conv = []
    output_of_the_GAP = []
    Final_predictions = []

    your_model.eval()
    for batched_data in test_loader:  
      Grad_CAM_Test_One_Before_Last_Conv, Grad_CAM_Test_Last_Conv, Grad_CAM_Test_GAP, Grad_CAM_Test_out = your_model(batched_data.x, batched_data.edge_index, batched_data.batch)
      #index_of_winner_label = CAM_output.argmax(dim=1) 
      logits = F.log_softmax(Grad_CAM_Test_out, dim=1)
      prob = F.softmax(logits, dim=1)
      index_of_winner_label = prob.argmax(dim=1) 

      index_of_winner_labels.append(index_of_winner_label.detach().tolist()) 
      Final_predictions.append(prob.detach().tolist())

      FeatureMaps_of_the_Last_Conv.append(Grad_CAM_Test_Last_Conv.detach().tolist())
    return FeatureMaps_of_the_Last_Conv, Final_predictions, index_of_winner_labels


  def Grad_CAM_Attribution_Scores(self, your_model, your_dataset):
    grads_wrt_graph_label, grads_wrt_class_zero, grads_wrt_class_one = self.compute_grad_cam_gradients(your_model, your_dataset)
    FeatureMaps_of_the_Last_Conv, Final_predictions, index_of_winner_labels = self.Grad_CAM_FeatureMAP_production(your_model, your_dataset)
    Grads_and_Maps_Multiplication_on_Nodes_of_each_graph = []
    Normalized_Attributions = []

    for i in range(len(FeatureMaps_of_the_Last_Conv)):
      Each_Graph = []
      for j in range(len(FeatureMaps_of_the_Last_Conv[i])):
        Each_Node = []
        for k in range(len(grads_wrt_graph_label[i])):
          Each_Node.append(np.multiply(grads_wrt_graph_label[i][k], FeatureMaps_of_the_Last_Conv[i][j]))
        Each_Graph.append(sum(np.mean(Each_Node, 0)))
      Grads_and_Maps_Multiplication_on_Nodes_of_each_graph.append(Each_Graph)
    #print(np.shape(Grads_and_Maps_Multiplication_on_Nodes_of_each_graph[0]))
    for i in range(len(FeatureMaps_of_the_Last_Conv)):
      Each_Graph = []
      for j in range(len(FeatureMaps_of_the_Last_Conv[i])):
        Each_Graph.append(Grads_and_Maps_Multiplication_on_Nodes_of_each_graph[i][j])
      norm = [(float(i))/(max(Each_Graph) + 1e-16) for i in Each_Graph] 
      Normalized_Attributions.append(norm)
    return Normalized_Attributions


  def drop_important_nodes(self, your_model, your_dataset, importance_range):
    Grad_CAM_attribution_scores = self.Grad_CAM_Attribution_Scores(your_model, your_dataset)
    occluded_GNNgraph_list = []
    
    for i in range(len(Grad_CAM_attribution_scores)):
      sample_graph = deepcopy(your_dataset[i])
      graph_dict ={}
      for j in range(len(sample_graph.x)):
        
        if self.is_salient((Grad_CAM_attribution_scores[i][j]), importance_range):
          #print("before: ", sample_graph.x[j])
          sample_graph.x[j][:] = 0
          #print(torch.zeros_like(sample_graph.x[j]))
          #print("manipulated: ",sample_graph.x[j])
          graph_dict[j] = True
        else:
          graph_dict[j] = False
      self.importance_dict[i] = graph_dict
      occluded_GNNgraph_list.append(sample_graph)
    return occluded_GNNgraph_list, Grad_CAM_attribution_scores#scipy.special.softmax(Grad_CAM_attribution_scores).tolist()


  


dataset = TUDataset(root='data/TUDataset', name='MUTAG')



#new_output = SA_GC(graph=[graph], importance_range=(0.5, 1), start_epoch=200, input_dim = len(graph.x[0]), hid_dim = 7, output_dim = 2)
new_output = Grad_CAM_GC(task="Graph Classification", method="Grad-CAM", graph=[dataset[0]], importance_range=(0.5, 1), load_index=200, input_dim = len(dataset[0].x[0]), hid_dim = 7, output_dim = 2)
print(new_output.new_graph[-1].x, dataset[-1].x)
print(new_output.saliency_maps)
print(new_output.importance_dict)


tensor([[1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]]) tensor([[1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1.

<ipython-input-41-b30f62c64275>:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Each_Graph.append(torch.tensor(item2.clone().detach().requires_grad_(True),requires_grad=True))


## **LRP GC Object Oriented Programming**

In [37]:
class LRP_GC(object):
  def __init__(self, task, method, graph, importance_range, load_index, input_dim, hid_dim, output_dim):

    self.GCN_model = self.load_model(task, method, load_index=load_index, input_dim=input_dim, hid_dim=hid_dim, output_dim=output_dim)
    self.criterion = torch.nn.CrossEntropyLoss()
    self.importance_dict = {}
    self.new_graph, self.saliency_maps = self.drop_important_nodes(self.GCN_model, graph, importance_range)
    


  def load_model(self, task, method, load_index, input_dim, hid_dim, output_dim):

    if load_index != 0:
      GCN_model, optimizer, load_index = self.loading_config(task, method, load_index, input_dim, hid_dim, output_dim)
      return GCN_model
    else:
      GCN_model = gcn_2l_model.GCN_2Layer_Model(model_level='graph', dim_node=input_dim, dim_hidden=hid_dim, dim_output=output_dim)
      return GCN_model


  def loading_config(self, task, method, load_index, input_dim, hid_dim, output_dim):
    GCN_model = gcn_2l_model.GCN_2Layer_Model(model_level='graph', dim_node=input_dim, dim_hidden=hid_dim, dim_output=output_dim)
    optimizer = torch.optim.Adam(params=GCN_model.parameters(), lr=0.001)
    checkpoint = torch.load("/content/drive/My Drive/Explainability Methods/"+str(method)+" on " + str(task) + "/Model/" + str(method) + " on " + str(task) + " classifier model_" + str(load_index)+".pt")
    GCN_model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    
    return GCN_model, optimizer, epoch
  

  def loss_calculations(self, preds, gtruth):
    loss_per_epoch = self.criterion(preds, gtruth)
    return loss_per_epoch
  
  def accumulate_weights(self, model_for_you):

    gconv1_weight = model_for_you.gconv1.weight.detach().tolist()

    gconv2_weight = model_for_you.gconv2.weight.detach().tolist()

    ffn_weight = model_for_you.ffn.weight.detach().tolist()

    return gconv1_weight, gconv2_weight, ffn_weight

  def GCN_Model_LRP(self, your_model, test_dataset):
    FFN_activations = []
    GConv2_activations = [] 
    GConv1_activations = [] 

    your_model.eval()
    for batch_of_graphs in test_dataset:
      post_conv1_test_activations, post_conv2_test_activations, lrp_gap, GCN_Model_test_activations = your_model(batch_of_graphs.x, batch_of_graphs.edge_index, batch_of_graphs.batch)
      GConv1_activations.append(post_conv1_test_activations.detach().tolist())
      GConv2_activations.append(post_conv2_test_activations.detach().tolist())
      FFN_activations.append(GCN_Model_test_activations.detach().tolist())  
      
    return GConv1_activations, GConv2_activations, FFN_activations 

  def my_relu(self, input):
    return np.maximum(0, input)

  def Compute_R_J(self, epsilon, preceding_layer_activations, exceding_layer_weights, next_layer_R_k, is_it_last):
    Denominators = []
    for i in range(len(preceding_layer_activations)):
      SUM_Denominator = 0
      for j in range(len(preceding_layer_activations[i])):
        for k in range(len(exceding_layer_weights[j])):
          SUM_Denominator = SUM_Denominator + preceding_layer_activations[i][j] * self.my_relu(exceding_layer_weights[j][k])
        Denominators.append(epsilon + SUM_Denominator)
    #print("Denominator is done: ", Denominators)
    #print("Denominator Length: ", len(Denominators))
    
    if is_it_last:
      Numerators = []
      for i in range(len(preceding_layer_activations)):
        Node_Numerator = []
        for j in range(len(preceding_layer_activations[i])):
          SUM_Nominator = 0
          for k in range(len(exceding_layer_weights[j])):
            SUM_Nominator = SUM_Nominator + preceding_layer_activations[i][j] * self.my_relu(exceding_layer_weights[j][k]) * next_layer_R_k[k]
          Node_Numerator.append(SUM_Nominator)
        Numerators.append(Node_Numerator)
    else:
      Numerators = []
      for i in range(len(preceding_layer_activations)):
        Node_Numerator = []
        for j in range(len(preceding_layer_activations[i])):
          SUM_Nominator = 0
          for k in range(len(exceding_layer_weights[j])):
            SUM_Nominator = SUM_Nominator + preceding_layer_activations[i][j] * self.my_relu(exceding_layer_weights[j][k]) * next_layer_R_k[i][k]
          Node_Numerator.append(SUM_Nominator)
        Numerators.append(Node_Numerator)
    
    R_Js_Graph = []
    for i in range(len(Numerators)):
      R_Js_Graph.append([x / Denominators[i] for x in Node_Numerator])

    return R_Js_Graph
  
  def Compute_R_K(self, FFN_activations, wrt):
    #print(FFN_activations)
    FFN_activations = FFN_activations[0]
    if wrt == 2:  #.     Graph Label
      last_layer_R_k = [0] * len(FFN_activations)
      last_layer_R_k[FFN_activations.index(max(FFN_activations))] = FFN_activations[FFN_activations.index(max(FFN_activations))]
      #print(last_layer_R_k)
      return last_layer_R_k
    elif wrt == 0: #.    Class 0
      last_layer_R_k = [0] * len(FFN_activations)
      last_layer_R_k[0] = FFN_activations[0]
      #print(last_layer_R_k)
      return last_layer_R_k
    elif wrt == 1: #.    Class 1
      last_layer_R_k = [0] * len(FFN_activations)
      last_layer_R_k[1] = FFN_activations[1]
      #print(last_layer_R_k)
      return last_layer_R_k

  def One_Graph_LRP(self, epsilon, wrt, graph_sample, weights, activations):
    GConv1_weight = weights[0]
    GConv1_weight_T = weights[1]

    GConv2_weight = weights[2]
    GConv2_weight_T = weights[3]

    FFN_weight = weights[4]
    FFN_weight_T = weights[5]

    GConv1_activations = activations[0]
    GConv2_activations = activations[1]
    FFN_activations = activations[2]

    last_layer_R_k = self.Compute_R_K(FFN_activations, wrt)
    #print("R_K: ",last_layer_R_k)
    R_J_hidden2 = self.Compute_R_J(epsilon, GConv2_activations, FFN_weight_T, last_layer_R_k, True)
    #print(R_J_hidden2)
    #print(len(R_J_hidden2))
    R_J_hidden1 = self.Compute_R_J(epsilon, GConv1_activations, GConv2_weight, R_J_hidden2, False)
    #print(R_J_hidden1)
    #print(len(R_J_hidden1))
    R_J_input = self.Compute_R_J(epsilon, graph_sample, GConv1_weight, R_J_hidden1, False)
    #print(R_J_input)
    #print(len(R_J_input))  
    
    
    return R_J_input


  def Normalize_LRPs(self, your_model, dataset):
    GConv1_activations, GConv2_activations, FFN_activations  = self.GCN_Model_LRP(your_model, dataset)
    GConv1_weight, GConv2_weight, FFN_weight = self.accumulate_weights(your_model)
    
    GConv1_weight_T = np.array(GConv1_weight).transpose()
    GConv1_weight_T = GConv1_weight_T.tolist()

    GConv2_weight_T = np.array(GConv2_weight).transpose()
    GConv2_weight_T = GConv2_weight_T.tolist()

    FFN_weight_T = np.array(FFN_weight).transpose()
    FFN_weight_T = FFN_weight_T.tolist()

    epsilon = 1e-16
    LRPs_Testset = []
    for i in range(len(dataset)):
      post_conv1_test_activations, post_conv2_test_activations, lrp_gap, GCN_Model_test_activations = your_model(dataset[i].x, dataset[i].edge_index, dataset[i].batch)
      wrt = GCN_Model_test_activations.argmax(dim=1).detach().tolist()[0] 
      #print(wrt)
      LRPs_Testset.append(self.One_Graph_LRP(epsilon, wrt, dataset[i].x.detach().tolist(), [GConv1_weight, GConv1_weight_T, GConv2_weight, GConv2_weight_T, FFN_weight, FFN_weight_T], [GConv1_activations[i], GConv2_activations[i], FFN_activations[i]]))
    
    Normalized_LRPs = []
    for i in range(len(LRPs_Testset)):
      Each_Graph = []
      for j in range(len(LRPs_Testset[i])):
        Each_Graph.append(sum(LRPs_Testset[i][j]))
      norm = [(float(i))/(max(Each_Graph) + 1e-16) for i in Each_Graph] 
      Normalized_LRPs.append(norm)
    print(LRPs_Testset[0])
    print(Normalized_LRPs[0])
    return Normalized_LRPs

  def is_salient(self, score, importance_range):
    start = importance_range[0]
    end = importance_range[1]
    #print(start, score, end)
    if float(start) <= float(score) <= float(end):
      return True
    else:
      return False


  def drop_important_nodes(self, your_model, your_dataset, importance_range):
    LRP_attribution_scores = self.Normalize_LRPs(your_model, your_dataset)
    occluded_GNNgraph_list = []
    
    for i in range(len(LRP_attribution_scores)):
      sample_graph = deepcopy(your_dataset[i])
      graph_dict ={}
      for j in range(len(sample_graph.x)):
        
        if self.is_salient((LRP_attribution_scores[i][j]), importance_range):
          #print("before: ", sample_graph.x[j])
          sample_graph.x[j][:] = 0
          #print(torch.zeros_like(sample_graph.x[j]))
          #print("manipulated: ",sample_graph.x[j])
          graph_dict[j] = True
        else:
          graph_dict[j] = False
      self.importance_dict[i] = graph_dict
      occluded_GNNgraph_list.append(sample_graph)
    return occluded_GNNgraph_list, LRP_attribution_scores#scipy.special.softmax(LRP_attribution_scores).tolist()




dataset = TUDataset(root='data/TUDataset', name='MUTAG')



new_output = LRP_GC(task="Graph Classification", method="LRP", graph=[dataset[0]], importance_range=(0.5, 1), load_index=200, input_dim = len(dataset[0].x[0]), hid_dim = 7, output_dim = 2)
print(new_output.new_graph[-1].x, dataset[-1].x)
print(new_output.saliency_maps)
print(new_output.importance_dict)


[[0.0, 0.0, -0.0627912565780123, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, -0.0627912565780123, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, -0.0627912565780123, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, -0.0627912565780123, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, -0.0627912565780123, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, -0.0627912565780123, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, -0.0627912565780123, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, -0.0627912565780123, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, -0.0627912565780123, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, -0.0627912565780123, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, -0.0627912565780123, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, -0.0627912565780123, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, -0.0627912565780123, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, -0.0627912565780123, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, -0.0627912565780123, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, -0.0627912565780123, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, -0.0627912565780123, 0.0, 0.0, 0.0, 0.0]]
[1.0000000000000016, 1.0000000000000016, 1.0000000000000016, 1.0000000000000016, 1.000000000000001

## **ExcitationBP GC Object Oriented Programming**

In [36]:
class ExcitationBP_GC(object):
  def __init__(self, task, method, graph, importance_range, load_index, input_dim, hid_dim, output_dim):

    self.GCN_model = self.load_model(task, method, load_index=load_index, input_dim=input_dim, hid_dim=hid_dim, output_dim=output_dim)
    self.criterion = torch.nn.CrossEntropyLoss()
    self.importance_dict = {}
    self.new_graph, self.saliency_maps = self.drop_important_nodes(self.GCN_model, graph, importance_range)
    


  def load_model(self, task, method, load_index, input_dim, hid_dim, output_dim):

    if load_index != 0:
      GCN_model, optimizer, load_index = self.loading_config(task, method, load_index, input_dim, hid_dim, output_dim)
      return GCN_model
    else:
      GCN_model = gcn_2l_model.GCN_2Layer_Model(model_level='graph', dim_node=input_dim, dim_hidden=hid_dim, dim_output=output_dim)
      return GCN_model


  def loading_config(self, task, method, load_index, input_dim, hid_dim, output_dim):
    GCN_model = gcn_2l_model.GCN_2Layer_Model(model_level='graph', dim_node=input_dim, dim_hidden=hid_dim, dim_output=output_dim)
    optimizer = torch.optim.Adam(params=GCN_model.parameters(), lr=0.001)
    checkpoint = torch.load("/content/drive/My Drive/Explainability Methods/"+str(method)+" on " + str(task) + "/Model/" + str(method) + " on " + str(task) + " classifier model_" + str(load_index)+".pt")
    GCN_model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    
    return GCN_model, optimizer, epoch
  

  def loss_calculations(self, preds, gtruth):
    loss_per_epoch = self.criterion(preds, gtruth)
    return loss_per_epoch

  def my_relu(self,input):
    return np.maximum(0, input)

  def accumulate_weights(self, model_for_you):
    gconv1_weight = model_for_you.gconv1.weight.detach().tolist()

    gconv2_weight = model_for_you.gconv2.weight.detach().tolist()

    ffn_weight = model_for_you.ffn.weight.detach().tolist()

    return gconv1_weight, gconv2_weight, ffn_weight
    
  def Division_by_Zero(self, epsilon, act_hat):
    for i in range(len(act_hat)):
      for j in range(len(act_hat[i])):
        if act_hat[i][j] == 0:
          act_hat[i][j] = act_hat[i][j] + epsilon
        else:
          act_hat[i][j] = act_hat[i][j]
    return act_hat


  def Compute_Pobabilities(self, last_layer, epsilon, preceding_layer_activations, exceding_layer_weights, exceding_layer_prob):


    # 1 Weights and Activations
    weights_and_activations_Graph = []
    for i in range(len(exceding_layer_weights)):
      weights_and_activations_Node = []
      for j in range(len(preceding_layer_activations)):
        weights_and_activations_Node.append(sum(np.multiply(self.my_relu(exceding_layer_weights[i]), preceding_layer_activations[j])))
      weights_and_activations_Graph.append(weights_and_activations_Node)
    #print(np.shape(weights_and_activations_Graph), weights_and_activations_Graph)
    #print("1 Multiplication of Weights and Activations: ", np.shape(weights_and_activations_Graph))
    weights_and_activations_Graph = self.Division_by_Zero(epsilon, weights_and_activations_Graph)
    weights_and_activations_Graph = np.array(weights_and_activations_Graph).transpose()
    weights_and_activations_Graph = weights_and_activations_Graph.tolist()
    #print("mul: ",np.shape(weights_and_activations_Graph), weights_and_activations_Graph)
    # 2 Point-Wise division
    #print(exceding_layer_prob)
    if last_layer:
      division_result_Graph = []
      for i in range(len(exceding_layer_prob)):
        division_result = [float(exceding_layer_prob[i]/ weights_and_activations_Graph[j][i]) for j in range(len(weights_and_activations_Graph))]
        division_result_Graph.append(division_result)
      #print("2 Division Results: ", np.shape(division_result_Graph))
      division_result_Graph = np.array(division_result_Graph).transpose()
      division_result_Graph = division_result_Graph.tolist()
      #print(np.shape(division_result_Graph), division_result_Graph)
    else:
      division_result_Graph = []
      for i in range(len(exceding_layer_prob)):
        division_result = [float(exceding_layer_prob[i][j]/ weights_and_activations_Graph[i][j]) for j in range(len(weights_and_activations_Graph[i]))]
        division_result_Graph.append(division_result)
      #print("2 Division Results: ", np.shape(division_result_Graph))
      #print(np.shape(division_result_Graph), division_result_Graph)

    # 3 Multiplication by Weights
    exceding_layer_weights = np.array(exceding_layer_weights).transpose()
    exceding_layer_weights = exceding_layer_weights.tolist()

    weights_third_step_Graph = []
    for i in range(len(division_result_Graph)):
      weights_third_step = []
      for j in range(len(exceding_layer_weights)):
        weights_third_step.append(sum([x*y for x,y in zip(self.my_relu(exceding_layer_weights[j]), division_result_Graph[i])]))
      weights_third_step_Graph.append(weights_third_step)
    #print("3 Third Step: ", np.shape(weights_third_step_Graph))
    #print(weights_third_step_Graph)

    # 4 Forth Step
    final_probability_Graph = []
    for i in range(len(preceding_layer_activations)):
      final_probability_vector = [preceding_layer_activations[i][j] * weights_third_step_Graph[i][j] for j in range(len(preceding_layer_activations[i]))]
      final_probability_Graph.append(final_probability_vector)
    #print(np.shape(final_probability_Graph))
    #print(final_probability_Graph)


    
    return final_probability_Graph

  def is_salient(self, score, importance_range):
    start = importance_range[0]
    end = importance_range[1]
    #print(start, score, end)
    if float(start) <= float(score) <= float(end):
      return True
    else:
      return False

  def normalize_labels_to_probabilistics(self, labels):
    labels = torch.tensor(labels)
    logits = F.log_softmax(labels, dim=1)
    prob = F.softmax(logits, dim=1)
    prob = prob.detach().tolist()
    norms = prob[0]
    #print(norms)
    return norms

  def Compute_P_last_layer(self, FFN_activations, wrt):
    prob = self.normalize_labels_to_probabilistics(FFN_activations)
    
    if wrt == 2:
      last_layer_R_k = [0] * len(prob)
      last_layer_R_k[prob.index(max(prob))] = max(prob)#1
      return last_layer_R_k

    elif wrt == 1:
      last_layer_R_k = [0] * len(prob)
      last_layer_R_k[1] = 1#prob[1]
      return last_layer_R_k

    elif wrt == 0:
      last_layer_R_k = [0] * len(prob)
      last_layer_R_k[0] = 1#prob[0]
      return last_layer_R_k


  def One_Graph_EB(self, wrt, epsilon, input_sample, weights, activations):
    GConv1_weight = weights[0]
    GConv1_weight_T = weights[1]

    GConv2_weight = weights[2]
    GConv2_weight_T = weights[3]

    FFN_weight = weights[4]
    FFN_weight_T = weights[5]

    GConv1_activations = activations[0]
    GConv2_activations = activations[1]
    FFN_activations = activations[2]

    #print(FFN_activations)
    last_layer_prob = self.Compute_P_last_layer(FFN_activations, wrt)
    #print("FFN: ", last_layer_prob)

    hidden2_probability_vector = self.Compute_Pobabilities(True, epsilon, GConv2_activations, FFN_weight, last_layer_prob)
    #print("Second GCN Probability: ",hidden2_probability_vector)
    #print("hid2: ", np.shape(hidden2_probability_vector)) 

    hidden1_probability_vector = self.Compute_Pobabilities(False, epsilon, GConv1_activations, GConv2_weight_T, hidden2_probability_vector)
    #print("First GCN Probability: ",hidden2_probability_vector)
    #print("hid1: ", np.shape(hidden1_probability_vector))
    


    input_probability_vector = self.Compute_Pobabilities(False, epsilon, input_sample, GConv1_weight_T, hidden1_probability_vector)
    #print("Input Probability: ",input_probability_vector)
    #print("input: ", np.shape(input_probability_vector))
    
    
    return input_probability_vector



  def transpose_weights(self, GConv1_weight, GConv2_weight, FFN_weight):
    GConv1_weight_T = np.array(GConv1_weight).transpose()
    GConv1_weight_T = GConv1_weight_T.tolist()

    GConv2_weight_T = np.array(GConv2_weight).transpose()
    GConv2_weight_T = GConv2_weight_T.tolist()

    FFN_weight_T = np.array(FFN_weight).transpose()
    FFN_weight_T = FFN_weight_T.tolist()

    return GConv1_weight_T, GConv2_weight_T, FFN_weight_T

  def Get_ExcitationBPs(self, your_model, dataset):
    GConv1_weight, GConv2_weight, FFN_weight = self.accumulate_weights(your_model)
    GConv1_weight_T, GConv2_weight_T, FFN_weight_T = self.transpose_weights(GConv1_weight, GConv2_weight, FFN_weight)

    GConv1_weight = GConv1_weight.copy()
    GConv2_weight = GConv2_weight.copy()
    FFN_weight = FFN_weight.copy()

    EBs_Testset = []
    epsilon = 1e-16
    #wrt = 0
    for i, graph in enumerate(dataset):
      post_conv1_test_activations, post_conv2_test_activations, lrp_gap, GCN_Model_test_activations = your_model(graph.x, graph.edge_index, graph.batch)
      wrt = GCN_Model_test_activations.argmax(dim=1).detach().tolist()[0]

      GCN_Model_test_activations = GCN_Model_test_activations.detach().tolist()
      post_conv2_test_activations = post_conv2_test_activations.detach().tolist()
      post_conv1_test_activations = post_conv1_test_activations.detach().tolist()

      EBs_Graph = self.One_Graph_EB(wrt, epsilon, dataset[i].x.detach().tolist(), [GConv1_weight, GConv1_weight_T, GConv2_weight, GConv2_weight_T, FFN_weight, FFN_weight_T], [post_conv1_test_activations, post_conv2_test_activations, GCN_Model_test_activations])
      graphs = []
      for j in range(len(EBs_Graph)):
        graphs.append(sum(EBs_Graph[j]))
      norm = [(float(i))/(max(graphs) - min(graphs) + 1e-16) for i in graphs] 
      EBs_Testset.append(norm)

    return EBs_Testset



  def drop_important_nodes(self, your_model, your_dataset, importance_range):
    EBP_attribution_scores = self.Get_ExcitationBPs(your_model, your_dataset)
    occluded_GNNgraph_list = []
    
    for i in range(len(EBP_attribution_scores)):
      sample_graph = deepcopy(your_dataset[i])
      graph_dict ={}
      for j in range(len(sample_graph.x)):
        
        if self.is_salient((EBP_attribution_scores[i][j]), importance_range):
          #print("before: ", sample_graph.x[j])
          sample_graph.x[j][:] = 0
          #print(torch.zeros_like(sample_graph.x[j]))
          #print("manipulated: ",sample_graph.x[j])
          graph_dict[j] = True
        else:
          graph_dict[j] = False
      self.importance_dict[i] = graph_dict
      occluded_GNNgraph_list.append(sample_graph)
    return occluded_GNNgraph_list, EBP_attribution_scores#scipy.special.softmax(EBP_attribution_scores).tolist()




dataset = TUDataset(root='data/TUDataset', name='MUTAG')



new_output = ExcitationBP_GC(task="Graph Classification", method="ExcitationBP", graph=[dataset[0]], importance_range=(0.5, 1), load_index=200, input_dim = len(dataset[0].x[0]), hid_dim = 7, output_dim = 2)
print(new_output.new_graph[-1].x, dataset[-1].x)
print(new_output.saliency_maps)
print(new_output.importance_dict)


tensor([[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.]]) tensor([[1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1.

In [34]:
import scipy
from scipy import special
scipy.special.softmax([1,2])

array([0.26894142, 0.73105858])